# 🗺️ Interactive Mapping - Athabasca Glacier
## MODIS Data Visualization with Date Selectors

This notebook provides interactive maps for exploring MODIS albedo data over the Athabasca Glacier with calendar date pickers and real-time updates.

---

## ⚠️ Prerequisites

**Run the setup notebook first**: `01_Setup_and_Configuration.ipynb`

Or run the setup cell below:

In [ ]:
# Quick setup - run this if you haven't run the setup notebook
%run 01_Setup_and_Configuration.ipynb

## 🛠️ Mapping Functions

Define functions for creating interactive maps with MODIS data.

In [ ]:
def mask_modis_snow_albedo(image):
    """Apply quality masking to MODIS snow albedo data"""
    albedo = image.select('Snow_Albedo_Daily_Tile')
    qa = image.select('NDSI_Snow_Cover_Basic_QA')
    
    # Quality and value filters
    valid_albedo = albedo.gte(5).And(albedo.lte(99))
    good_quality = qa.lte(DEFAULT_QA_THRESHOLD)  # QA ≤ 1
    
    # Scale factor (MODIS albedo is stored as integers 0-100)
    scaled = albedo.multiply(0.01).updateMask(valid_albedo.And(good_quality))
    
    return scaled.rename('albedo_daily').copyProperties(image, ['system:time_start'])

def get_modis_snow_albedo(date_str):
    """Get MODIS snow albedo data for a specific date"""
    
    # Date range (±1 day for better coverage)
    start_date = ee.Date(date_str).advance(-1, 'day')
    end_date = ee.Date(date_str).advance(1, 'day')
    
    # Get Terra and Aqua collections
    terra_collection = ee.ImageCollection(MODIS_COLLECTIONS['snow_terra']) \
        .filterBounds(athabasca_roi) \
        .filterDate(start_date, end_date)
    
    aqua_collection = ee.ImageCollection(MODIS_COLLECTIONS['snow_aqua']) \
        .filterBounds(athabasca_roi) \
        .filterDate(start_date, end_date)
    
    # Apply masking and combine
    terra_masked = terra_collection.map(mask_modis_snow_albedo)
    aqua_masked = aqua_collection.map(mask_modis_snow_albedo)
    
    # Merge collections and create mosaic
    combined = terra_masked.merge(aqua_masked)
    
    if combined.size().getInfo() > 0:
        return combined.mosaic()
    else:
        return None

def create_valid_pixel_boundaries(date_str):
    """Create boundaries for MODIS pixels with valid data"""
    
    modis_data = get_modis_snow_albedo(date_str)
    
    if modis_data is not None:
        # Clip to glacier and convert to vectors
        glacier_data = modis_data.select('albedo_daily').clip(athabasca_roi)
        
        pixel_vectors = glacier_data.reduceToVectors(
            geometry=athabasca_roi,
            scale=DEFAULT_SCALE,
            geometryType='polygon',
            eightConnected=False,
            maxPixels=1e6,
            bestEffort=True
        )
        
        return pixel_vectors
    else:
        return ee.FeatureCollection([])

print("✅ Mapping functions defined!")

## 🗺️ Basic Interactive Map

Create a basic map showing the glacier extent and MODIS data for a specific date.

In [ ]:
def create_glacier_map(date_str='2023-08-15', show_pixels=True, show_albedo=True):
    """Create an interactive map of the glacier with MODIS data"""
    
    # Get glacier center for map positioning
    try:
        glacier_centroid = athabasca_roi.centroid().getInfo()
        center_lat = glacier_centroid['coordinates'][1]
        center_lon = glacier_centroid['coordinates'][0]
    except:
        center_lat, center_lon = 52.174, -117.287  # Fallback coordinates
    
    # Create map
    Map = geemap.Map(center=[center_lat, center_lon], zoom=13)
    
    # Add glacier boundary
    glacier_style = {
        'color': '#ff0000',
        'fillColor': '#ff0000',
        'weight': 3,
        'fillOpacity': 0.1
    }
    
    glacier_fc = ee.FeatureCollection([ee.Feature(athabasca_roi, {'name': 'Athabasca Glacier'})])
    Map.add_ee_layer(glacier_fc, glacier_style, 'Glacier Boundary', True)
    
    # Add MODIS albedo data
    if show_albedo:
        modis_data = get_modis_snow_albedo(date_str)
        if modis_data is not None:
            albedo_vis = {
                'min': 0.0,
                'max': 1.0,
                'palette': ['0d0887', '6a00a8', 'b12a90', 'e16462', 'fca636', 'f0f921']
            }
            
            Map.add_ee_layer(
                modis_data.select('albedo_daily').clip(athabasca_roi),
                albedo_vis,
                f'Snow Albedo ({date_str})',
                True
            )
    
    # Add valid pixel boundaries
    if show_pixels:
        try:
            valid_pixels = create_valid_pixel_boundaries(date_str)
            pixel_style = {
                'color': '#0066cc',
                'fillColor': 'transparent',
                'weight': 2,
                'fillOpacity': 0
            }
            
            Map.add_ee_layer(valid_pixels, pixel_style, f'Valid Pixels ({date_str})', True)
            
            # Get pixel count
            pixel_count = valid_pixels.size().getInfo()
            print(f"📊 Found {pixel_count} valid MODIS pixels for {date_str}")
            
        except Exception as e:
            print(f"⚠️ Could not create pixel boundaries: {e}")
    
    # Add weather station
    Map.add_ee_layer(
        ee.Geometry.Point(ATHABASCA_STATION),
        {'color': 'yellow'},
        'Weather Station',
        True
    )
    
    return Map

# Create initial map
print("🗺️ Creating initial map...")
initial_map = create_glacier_map()
initial_map

## 🎛️ Interactive Map with Date Picker

Create an interactive map with calendar widgets for date selection and real-time updates.

In [ ]:
# Create interactive widgets
date_picker = widgets.DatePicker(
    description='📅 Select Date:',
    value=date(2023, 8, 15),
    disabled=False,
    style={'description_width': '120px'}
)

show_pixels_checkbox = widgets.Checkbox(
    value=True,
    description='Show Valid Pixels',
    disabled=False,
    style={'description_width': '120px'}
)

show_albedo_checkbox = widgets.Checkbox(
    value=True,
    description='Show Albedo Data',
    disabled=False,
    style={'description_width': '120px'}
)

update_button = widgets.Button(
    description='🔄 Update Map',
    button_style='primary',
    tooltip='Click to update map with selected date'
)

status_label = widgets.Label(value='Ready')
output_area = widgets.Output()

# Create the map container
map_output = widgets.Output()

# Initialize map
with map_output:
    current_map = create_glacier_map()
    display(current_map)

def update_map_interactive(b):
    """Update map when button is clicked"""
    with output_area:
        output_area.clear_output()
        status_label.value = 'Loading...'
        
        try:
            selected_date = date_picker.value.strftime('%Y-%m-%d')
            
            print(f"🔄 Updating map for {selected_date}...")
            
            # Clear and recreate map
            with map_output:
                map_output.clear_output()
                new_map = create_glacier_map(
                    date_str=selected_date,
                    show_pixels=show_pixels_checkbox.value,
                    show_albedo=show_albedo_checkbox.value
                )
                display(new_map)
            
            status_label.value = f'Updated: {selected_date}'
            
        except Exception as e:
            print(f"❌ Error updating map: {e}")
            status_label.value = 'Error occurred'

# Connect button to update function
update_button.on_click(update_map_interactive)

# Create control panel
controls = widgets.VBox([
    widgets.HTML("<h3 style='margin-bottom:10px;'>🎛️ Map Controls</h3>"),
    date_picker,
    show_pixels_checkbox,
    show_albedo_checkbox,
    widgets.HBox([update_button, status_label]),
    widgets.HTML("<hr style='margin:10px 0;'>"),
    output_area
], layout=widgets.Layout(width='300px', padding='10px'))

# Combine controls and map
interactive_map_widget = widgets.HBox([
    map_output,
    controls
], layout=widgets.Layout(width='100%'))

display(interactive_map_widget)

## 📅 MODIS Data Browser

Explore available MODIS data across different date ranges.

In [ ]:
# Date range browser widgets
start_date_picker = widgets.DatePicker(
    description='📅 Start Date:',
    value=date(2023, 6, 1),
    disabled=False,
    style={'description_width': '100px'}
)

end_date_picker = widgets.DatePicker(
    description='📅 End Date:',
    value=date(2023, 9, 30),
    disabled=False,
    style={'description_width': '100px'}
)

browse_button = widgets.Button(
    description='🔍 Browse Data',
    button_style='success'
)

browse_output = widgets.Output()

def browse_data_availability(b):
    """Browse available MODIS data in date range"""
    with browse_output:
        browse_output.clear_output()
        
        start_str = start_date_picker.value.strftime('%Y-%m-%d')
        end_str = end_date_picker.value.strftime('%Y-%m-%d')
        
        print(f"🔍 Searching MODIS data from {start_str} to {end_str}...")
        
        try:
            # Check Terra collection
            terra_collection = ee.ImageCollection(MODIS_COLLECTIONS['snow_terra']) \
                .filterBounds(athabasca_roi) \
                .filterDate(start_str, end_str)
            
            # Check Aqua collection  
            aqua_collection = ee.ImageCollection(MODIS_COLLECTIONS['snow_aqua']) \
                .filterBounds(athabasca_roi) \
                .filterDate(start_str, end_str)
            
            terra_count = terra_collection.size().getInfo()
            aqua_count = aqua_collection.size().getInfo()
            total_count = terra_count + aqua_count
            
            print(f"📊 DATA AVAILABILITY RESULTS:")
            print(f"   • Terra (MOD10A1): {terra_count} images")
            print(f"   • Aqua (MYD10A1): {aqua_count} images")
            print(f"   • Total: {total_count} images")
            
            if total_count > 0:
                # Calculate average per month
                start_dt = datetime.strptime(start_str, '%Y-%m-%d')
                end_dt = datetime.strptime(end_str, '%Y-%m-%d')
                months = (end_dt - start_dt).days / 30.44
                avg_per_month = total_count / months if months > 0 else 0
                
                print(f"   • Average: {avg_per_month:.1f} images/month")
                
                # Get some sample dates
                if terra_count > 0:
                    sample_terra = terra_collection.limit(5).aggregate_array('system:index').getInfo()
                    print(f"   • Sample Terra dates: {sample_terra}")
                
                print("✅ Data available for analysis!")
            else:
                print("⚠️ No data found in this date range")
                
        except Exception as e:
            print(f"❌ Error browsing data: {e}")

browse_button.on_click(browse_data_availability)

# Create browser widget
data_browser = widgets.VBox([
    widgets.HTML("<h3>📅 MODIS Data Availability Browser</h3>"),
    widgets.HBox([start_date_picker, end_date_picker]),
    browse_button,
    browse_output
])

display(data_browser)

## ⚡ Quick Date Selection

Buttons for quickly jumping to common dates of interest.

In [ ]:
# Quick date selection buttons
def create_quick_date_button(label, target_date, description=""):
    """Create a button for quick date selection"""
    button = widgets.Button(
        description=label,
        button_style='info',
        tooltip=description or f"Jump to {target_date}"
    )
    
    def on_click(b):
        date_picker.value = datetime.strptime(target_date, '%Y-%m-%d').date()
        # Auto-update map
        update_map_interactive(None)
    
    button.on_click(on_click)
    return button

# Create quick date buttons
quick_buttons = widgets.HBox([
    create_quick_date_button("Early Summer", "2023-06-15", "Early melt season"),
    create_quick_date_button("Mid Summer", "2023-07-15", "Peak melt period"),
    create_quick_date_button("Late Summer", "2023-08-15", "Maximum melt"),
    create_quick_date_button("Early Fall", "2023-09-15", "End of melt season"),
    create_quick_date_button("Winter", "2023-12-15", "Snow accumulation")
])

display(widgets.VBox([
    widgets.HTML("<h4>⚡ Quick Date Selection</h4>"),
    quick_buttons
]))

## 💾 Save Maps

Save your interactive maps as HTML files for sharing or offline viewing.

In [ ]:
def save_current_map():
    """Save the current map configuration as HTML"""
    selected_date = date_picker.value.strftime('%Y-%m-%d')
    
    # Create map with current settings
    map_to_save = create_glacier_map(
        date_str=selected_date,
        show_pixels=show_pixels_checkbox.value,
        show_albedo=show_albedo_checkbox.value
    )
    
    # Generate filename
    filename = f"athabasca_glacier_map_{selected_date.replace('-', '')}.html"
    
    # Save map
    map_to_save.to_html(filename)
    
    print(f"💾 Map saved as: {filename}")
    print(f"📂 Open this file in your web browser to view the interactive map")
    
    return filename

# Save button
save_button = widgets.Button(
    description='💾 Save Current Map',
    button_style='warning'
)

save_output = widgets.Output()

def on_save_click(b):
    with save_output:
        save_output.clear_output()
        save_current_map()

save_button.on_click(on_save_click)

display(widgets.VBox([
    widgets.HTML("<h4>💾 Save Map</h4>"),
    save_button,
    save_output
]))

## 💡 Usage Tips

### 🎯 Map Features:
- **Red boundary**: Exact glacier extent from GeoJSON mask
- **Blue outlines**: MODIS pixels with valid data (passed QA filters)
- **Color overlay**: Snow albedo values (purple=low, yellow=high)
- **Yellow marker**: Weather station location

### 🎛️ Interactive Controls:
- **Date Picker**: Click to open calendar and select any date
- **Checkboxes**: Toggle pixel boundaries and albedo overlay
- **Update Button**: Refresh map with new settings
- **Quick Buttons**: Jump to seasonal dates

### 📊 Data Quality:
- Only pixels with QA ≤ 1 are shown (good quality)
- Albedo values are scaled from 0.0 to 1.0
- Missing data appears as gaps in coverage

### 🔍 Best Practices:
- Try summer dates (June-September) for best data coverage
- Use the data browser to find periods with good availability
- Save interesting maps as HTML files for later reference

---

**Next**: Continue to `03_Data_Processing.ipynb` for time series extraction and analysis.